# Azure ML Workspace

## Importing libraries

In [20]:
from azureml.core import Workspace, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
import requests

## Initialise workspace
Initialise a workspace using the values from config.json
This Json file was downloaded after provisioning a workspace through the online interface.
This could also be created programmatically using Workspace.create()

In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
wspace = Workspace.from_config()

## Register the ML model
Load the ML model into the workspace from file.
This file was produced previously.
This method will allow different models to be loaded using the same workspace.

In [22]:
##  Load the ML model from file. This file will most likely be a pickle dump.

model = Model.register(workspace = wspace,
                       model_path = "models/knn.pkl",
                       model_name = "KNN",
                       description = "Music Genre Classification")

print("Model Loaded")

Registering model KNN
Model Loaded


## Create an environment
Create an environment for the model to run on the Azure servers.

In [23]:
environment = Environment('env')
##  Load conda dependencies required for this environment.
##  Register environment for the workspace

env = Environment("env")

condaDep = CondaDependencies.create(pip_packages = ["pandas == 1.1.3",
                                                    "azureml-core"],
                                    conda_packages = ["scikit-learn == 0.23.2"])

env.python.conda_dependencies = condaDep

env.register(workspace = wspace)

print("Environment Registered")

my_env = Environment.get(workspace = wspace, name = "env")
my_env.save_to_directory("./environ", overwrite = True)

print("Workspace Saved")

Environment Registered
Workspace Saved


## Provision the hardware to be used
Define the hardware configuration that will be used to process the model.

In [24]:
##  Define the hardware config and deployment config

azure_container_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1,
    description = "Genre Classication ML")

inf_config = InferenceConfig(entry_script = "ent_script.py",
                                   environment = my_env)

print("Azure Container Provisioned")

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


Azure Container Provisioned


## Deploy

In [25]:
service = Model.deploy(workspace = wspace,
                       name = "music-genre-classification",
                       models = [model],
                       inference_config = inf_config,
                       deployment_config = azure_container_config,
                       overwrite = True)

service.wait_for_deployment(show_output = True)
url = service.scoring_uri
print(url)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................................................................................................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 32b6f746-49d5-41b5-b71b-a677bd5afe0d
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: music-genre-classification. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 66425b6f967443bc9d0872cda5da0cca.azurecr.io/azureml/azureml_7dfe8dd778c74da1189b045f536704c1 locally. Please refer to https://aka.ms/debugimage#servic

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 32b6f746-49d5-41b5-b71b-a677bd5afe0d
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: music-genre-classification. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 66425b6f967443bc9d0872cda5da0cca.azurecr.io/azureml/azureml_7dfe8dd778c74da1189b045f536704c1 local

## Consume Endpoint

In [ ]:
head = {"Content-Type": "application/json"}
return_vals = requests.post(url, data = {"data": [[1,1,1,1]]}, headers = head)
print(return_vals.status_code)
print(return_vals.json())